<a href="https://colab.research.google.com/github/Cicciokr/latin-ai-model/blob/main/02B.Fine-Tuning-Model-Text-Gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install datasets
#!pip install evaluate
#!pip install rouge_score
!pip install wandb
!pip install transformers
#!pip install datasets==2.15.0
#!pip install evaluate==0.4.1
#!pip install accelerate==0.25.0
#!pip install tqdm==4.66.1
!pip install numpy
!pip install sklearn


import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoTokenizer, TrainerCallback, EarlyStoppingCallback, get_scheduler, BartForConditionalGeneration, BartTokenizer, AutoModelForSeq2SeqLM
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
#from numba import cuda
import math
import torch
import wandb
import os

os.environ["WANDB_PROJECT"] = "Training-BARTLA"
wandb.login(key="e115f18967efc277b293c042cb216e776d82a741")
wandb.init()

MODEL_NAME = "Cicciokr/BART-CC100-la"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trucantion=True, model_max_length=1024)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)


def preprocess_function(examples):
    tokens = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=1024)
    tokens["labels"] = tokens["input_ids"].copy()  # Aggiunge i labels uguali agli input_ids
    return tokens

#"/kaggle/input/the-latin-library/thelatinlibrary_cleaned.txt"
dataset = load_dataset("text", data_files="/content/drive/MyDrive/Colab Notebooks/dataset/thelatinlibrary_cleaned.txt")
dataset_split = dataset['train'].train_test_split(train_size=0.8, test_size=0.2, shuffle=True)
#dataset_split_train = dataset['train'].train_test_split(test_size=0.0001, shuffle=True)
# Applicare la tokenizzazione
tokenized_datasets_test = dataset_split['test'].map(preprocess_function, batched=True, num_proc=4)
tokenized_datasets_train = dataset_split['train'].map(preprocess_function, batched=True, num_proc=4)

tokenized_datasets_test=tokenized_datasets_test.remove_columns(["text"])
tokenized_datasets_test.set_format("torch")

tokenized_datasets_train=tokenized_datasets_train.remove_columns(["text"])
tokenized_datasets_train.set_format("torch")

print(tokenized_datasets_train)
print(tokenized_datasets_test)

#il 20% dei dati viene usato come test e l'80% viene usato come train, per evitare overfitting
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/BART",
    run_name="bart-la-s",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    auto_find_batch_size=True,
    fp16=True,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/Colab Notebooks/BART/logs",
    warmup_steps=10000,
    logging_steps=10000,
    save_total_limit=3,
    greater_is_better=True,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    report_to="wandb",
    metric_for_best_model="R2",
    dataloader_num_workers=4
)
#xm.rendezvous("start_training")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    tokenizer=tokenizer
)

trainer.train(resume_from_checkpoint="checkpoint-18416")

model.save_model("/content/drive/MyDrive/Colab Notebooks/BART/bart-la-s")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/BART/bart-la-s-tokenizer")

wandb.finish()


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'Cache' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
